In [0]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://repos.azul.com/zulu/deb stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-16 06:45:43--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  2.61MB/s    in 0.3s    

2023-03-16 06:45:44 (2.61 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [0]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [0]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "s3://bigdata-etl-randie/amazon_reviews_us_Baby_v1_00.tsv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(url, sep='\t', inferSchema=True)
df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [0]:
# Get the number of rows in the DataFrame.
df.count()

Out[57]: 1752932

# Transform the Data

## Examine the Schema

In [0]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Create the "review_id_table".

In [0]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_table = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8EWA1OFT84NX|    9970739|B00GSP5D94|     329991347| 2015-08-31|
|R2JWY4YRQD4FOP|   23538442|B00YYDDZGU|     646108902| 2015-08-31|
| RL5ESX231LZ0B|    8273344|B00BUBNZC8|     642922361| 2015-08-31|
| RRMS9ZWJ2KD08|   24557753|B00AWLZFTS|     494272733| 2015-08-31|
|R14I3ZG5E6S7YM|   46263340|B00KM60D3Q|     305813185| 2015-08-31|
|R13EPSFP5DODN5|   24557753|B00PQMRZG4|     607341708| 2015-08-31|
| R6RBP4HTE67SY|   33520065|B005DL5970|     971881542| 2015-08-31|
|R15B3EU40RSU2W|   20241560|B00C6D2WL4|      93827401| 2015-08-31|
| RP4DD53A4ZJA2|    9987983|B0083973FK|     958629336| 2015-08-31|
|R2C99DJEO4RZ4K|   52570308|B00RLYG2S2|     147324304| 2015-08-31|
| REV51EW323H8W|    9287389|B010UX9T5I|     446691106| 2015-08-31|
|R2GQ3W03WIUZKE|   32840762|B00VWBY7SC|     271204734| 2015-08

## Create the "products" Table

In [0]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00VWBY7SC|Bugzi Stroller Ho...|
|B00YYDDZGU|Pacifier Clip Gir...|
|B00KM60D3Q|Summer Infant Ult...|
|B005DL5970|Natural HE Powder...|
|B00M2F0OYS|Motorola Baby Mon...|
|B010UX9T5I|Lovinglove Baby G...|
|B00BUBNZC8|Udder Covers - Br...|
|B00GSP5D94|Summer Infant Swa...|
|B00PQMRZG4|Summer Infant Kee...|
|B0071D1AKI|Aqueduck The ORIG...|
|B0083973FK|Sposie Booster Pa...|
|B00J0YTS1E|My Natural Owl Mu...|
|B00C6D2WL4|Dr. Brown's Bottl...|
|B005BIOOYO|Flip Stay-Dry Ins...|
|B00AWLZFTS|Gerber Graduates ...|
|B00UGV8BEU|Baby Bandana Bibs...|
|B00SN6F9NG|Rhoost Nail Clipp...|
|B00RLYG2S2|Abiie Beyond Wood...|
|B0070U6R9Q|NUK Hello Kitty S...|
|B00QCBD5AS|Minnie Mouse Delu...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [0]:
from pyspark.sql.functions import count
from pyspark.sql.functions import desc

# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupBy("customer_id").agg(count("*").alias("customer_count"))
customers_df = customers_df.orderBy(desc("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12084007|           192|
|   51591392|           141|
|   49633579|           140|
|   52457037|           114|
|   13103726|           112|
|   51983887|           109|
|   31461331|           105|
|   46106653|           105|
|   46137405|           105|
|   14389539|           102|
|   52097584|           101|
|   43715603|            99|
|   13404682|            98|
|   45725380|            98|
|   48428870|            93|
|   31546953|            90|
|   38555595|            89|
|   46912614|            87|
|   48384094|            86|
|   44893973|            85|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [0]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8EWA1OFT84NX|          5|            0|          0|   N|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|
| RL5ESX231LZ0B|          5|            0|          0|   N|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|
|R13EPSFP5DODN5|          4|            0|          0|   N|
| R6RBP4HTE67SY|          5|            0|          0|   N|
|R15B3EU40RSU2W|          5|            0|          0|   N|
| RP4DD53A4ZJA2|          5|            0|          0|   N|
|R2C99DJEO4RZ4K|          5|            3|          4|   N|
| REV51EW323H8W|          5|            0|          0|   N|
|R2GQ3W03WIUZKE|          5|            0|          0|   N|
| RTI1YI7K6GE3D|          5|            0|          0|   N|
|R3V9C2C0SPSZU6|          5|            

# Load

In [0]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpointredactedforsecurity>:5432/rds_database"
config = {"user":"postgres", "password": "password", "driver":"org.postgresql.Driver"}

In [0]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-2977815509487994> in <module>
      1 # Write review_id_df to table in RDS
----> 2 review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

/databricks/spark/python/pyspark/sql/readwriter.py in jdbc(self, url, table, mode, properties)
   1470         for k in properties:
   1471             jprop.setProperty(k, properties[k])
-> 1472         self.mode(mode)._jwrite.jdbc(url, table, jprop)
   1473 
   1474 

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    115     def deco(

In [0]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [0]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [0]:
# Write vine_df to table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)